# Stock Strayegy - High Frequency 0003 - Intraday Trading Strategy

# 1. Import Library

In [1]:
import dai
import random
import pandas as pd

# 2. Prepare Strategy Data

In [2]:
sd = "2026-01-01 00:00:00.000"
ed = "2026-02-13 23:59:59.000"

In [3]:
sql_factor =  f"""
SELECT 
    date, 
    instrument,
    float_market_cap AS factor,
FROM cn_stock_prefactors
WHERE is_risk_warning = 0
AND list_sector IN (1, 2, 3)
AND list_days > 365
ORDER BY date, factor
"""

In [4]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 5
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [5]:
df_strategy = dai.query(sql_trade, filters={"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2026-01-05,001260.SZ,640080000.0,1.0,0.2
1,2026-01-05,603307.SH,680680000.0,2.0,0.2
2,2026-01-05,301314.SZ,715000000.0,3.0,0.2
3,2026-01-05,301287.SZ,730474601.5,4.0,0.2
4,2026-01-05,001366.SZ,736089600.0,5.0,0.2
...,...,...,...,...,...
145,2026-02-13,603307.SH,705670000.0,1.0,0.2
146,2026-02-13,001260.SZ,725170000.0,2.0,0.2
147,2026-02-13,301287.SZ,775267052.0,3.0,0.2
148,2026-02-13,301309.SZ,782629360.0,4.0,0.2


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    context.subscribe_bar(context.instruments, '1m')

def BigTrader_Handle_Tick(context, tick):
    pass

def build_trade_df(context, data):
    today = data.current_dt.strftime("%Y-%m-%d")
    df_today = context.data[context.data["date"] == today].copy()

    df_target = df_today[["instrument", "position"]].copy()
    df_target["position"] = df_target["position"].fillna(0.0).astype(float)
    df_target.rename(columns={"position": "tar_pos"}, inplace=True)

    positions = context.get_account_positions()
    df_cur = pd.DataFrame([(k, float(v.current_qty)) for k, v in positions.items()], columns=["instrument", "cur_pos"])
    if df_cur.empty:
        df_cur = pd.DataFrame(columns=["instrument", "cur_pos"])

    df_trade = pd.merge(df_cur, df_target, on="instrument", how="outer")

    df_trade["cur_pos"] = df_trade["cur_pos"].fillna(0.0)
    df_trade["tar_pos"] = df_trade["tar_pos"].fillna(0.0)

    def decide_action(row):
        if row.tar_pos > row.cur_pos:
            return "buy"
        elif row.tar_pos < row.cur_pos:
            return "sell"
        else:
            return "hold"

    df_trade["action"] = df_trade.apply(decide_action, axis=1)

    return df_trade

def BigTrader_Handle_Data(context, data):

    df_today = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]
    time_now = data.current_dt.strftime("%Y-%m-%d %H:%M:%S")
    
    if len(df_today) == 0 or time_now[11:] not in  ("09:35:00", "14:55:00"):
        return
    
    df_trade = build_trade_df(context, data)

    if time_now[11:] == "09:35:00":
        df_sell = df_trade[df_trade["action"] == "sell"]
        for _, r in df_sell.iterrows():
            context.order_target_percent(r["instrument"], float(r["tar_pos"]))
    
    if time_now[11:] == "14:55:00":
        df_buy = df_trade[df_trade["action"] == "buy"]
        for _, r in df_buy.iterrows():
            context.order_target_percent(r["instrument"], float(r["tar_pos"]))

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""minute""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-19 11:53:56] [info     ] bigtrader.v34 开始运行 ..
[2026-02-19 11:53:56] [info     ] 2026-01-05, 2026-02-13, , , instruments=6
[2026-02-19 11:53:56] [info     ] bigtrader module V2.2.0
[2026-02-19 11:53:56] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10


handle_order_new() trading_account process req failed for order=[bkt000,,301309.SZ,'1','0',0,100,29.08,14(accepted),20260112 09:35:00,16,strategy], available=258.7020969257792
handle_order_new() trading_account process req failed for order=[bkt000,,301309.SZ,'1','0',0,300,29.21,14(accepted),20260115 09:35:00,22,strategy], available=8554.66209692578
handle_order_new() trading_account process req failed for order=[bkt000,,603307.SH,'1','0',0,300,40.89,14(accepted),20260115 09:35:00,23,strategy], available=8554.66209692578


[2026-02-19 11:53:58] [info     ] backtest done, raw_perf_ds:dai.DataSource("_562108465e344d468b44b1a2d3059d6c")


[2026-02-19 11:53:58] [info     ] bigtrader.v34 运行完成 [2.527s].
